In [4]:
import pandas as pd

# Merge and filter data
-  Read all the sensor data
- Filter for only data from sensors present in our model
- Propperly read the timestamps
- Aggregate all the processed data into a new dataframe

In [2]:
# File names and relative paths
# files = ['../Dados/2013AEDL.csv', '../Dados/1S2014AEDL.csv', '../Dados/2S2014AEDL.csv', '../Dados/1P2015AEDL.csv', '../Dados/2P2015AEDL.csv']
files = ['./Dataset/2013AEDL.csv', './Dataset/1S2014AEDL.csv', './Dataset/2S2014AEDL.csv', './Dataset/1P2015AEDL.csv', './Dataset/2P2015AEDL.csv']

# Define the list of id of the sensors in our model
target_ids = [121726,121727,121731,121732,121733,121734,121735,121736,121741,121742,121754,121755,121756]

# Filter the DataFrame: we want to keep only rows where 'EQUIPMENTID' matches the sensor IDs we use
col = 'EQUIPMENTID'

# Create dataframe to concatenate all the useful information
filtered_df = pd.DataFrame()

for file in files:
    df = pd.read_csv(file)
    # df.info()

    # Convert data types before filtering
    df["AGG_PERIOD_START"] = pd.to_datetime(df["AGG_PERIOD_START"])
    df["LANE_BUNDLE_DIRECTION"] = df["LANE_BUNDLE_DIRECTION"].astype("string")
    
    # Filter for only rows with matching sensor ids
    matching_df = df[df[col].isin(target_ids)]
    
    filtered_df = pd.concat([filtered_df, matching_df], ignore_index=True)

    print(f"Original rows: {len(df)}")
    print(f"Filtered rows: {len(matching_df)}")
    # print(filtered_df.info())

filtered_df.info()

Original rows: 3890630
Filtered rows: 2063480
Original rows: 2251668
Filtered rows: 1153163
Original rows: 2745968
Filtered rows: 1312094
Original rows: 1315265
Filtered rows: 648009
Original rows: 3881466
Filtered rows: 1917689
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094435 entries, 0 to 7094434
Data columns (total 20 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   AGGREGATE_BY_LANE_BUNDLEID  int64         
 1   AGG_ID                      int64         
 2   EQUIPMENTID                 int64         
 3   AGG_PERIOD_START            datetime64[ns]
 4   AGG_PERIOD_LEN_MINS         int64         
 5   NR_LANES                    int64         
 6   LANE_BUNDLE_DIRECTION       string        
 7   TOTAL_VOLUME                int64         
 8   AVG_SPEED_ARITHMETIC        float64       
 9   AVG_SPEED_HARMONIC          float64       
 10  AVG_LENGTH                  float64       
 11  AVG_SPACING                 f

In [6]:
filtered_df.to_csv("./Dataset/filtered_data.csv", index=False)
# filtered_df.to_parquet('../Dados/filtered_data.parquet', index=False)

# Transform data for MAB

## Eliminate useless columns

In [5]:
# concatenated AEDL data
df = pd.read_csv("./Dataset/filtered_data.csv", parse_dates=["AGG_PERIOD_START"])
# df = filtered_df.copy()

In [6]:
half_complete_df = df.drop(columns=["AGGREGATE_BY_LANE_BUNDLEID","AGG_ID","AGG_PERIOD_LEN_MINS","NR_LANES","AVG_LENGTH","OCCUPANCY","VOLUME_CLASSE_A","VOLUME_CLASSE_B","VOLUME_CLASSE_C","VOLUME_CLASSE_D","VOLUME_CLASSE_0","AXLE_CLASS_VOLUMES"])
half_complete_df.head(3)
half_complete_df.to_csv("./Dataset/more_complete_data.csv", index=False)
# half_complete_df.to_csv("../Dados/more_complete_data.csv", index=False)
simple_df = half_complete_df.drop(columns=["AVG_SPEED_ARITHMETIC","AVG_SPEED_HARMONIC","AVG_SPACING","LIGHT_VEHICLE_RATE"])
simple_df.head(3)
simple_df.to_csv("./Dataset/simplest_data.csv", index=False)
# simple_df.to_csv("../Dados/simplest_data.csv", index=False)

## Create tags for arms
### (gather all data from the same sensor, the same time interval and the same direction)

In [14]:
# MAP DATA INTO BINS FOR EACH ARM
# Arm = (time_bin, sensor_id, direction)

# Minutes since midnight
simple_df["min_since_midnight"] = (simple_df["AGG_PERIOD_START"].dt.hour*60 + simple_df["AGG_PERIOD_START"].dt.minute)


# 30‑minute bins: [0..47]
simple_df["half_time_bin"] = (simple_df["min_since_midnight"] // 30).astype("int8")

# 5‑minute bins: [0..287]
simple_df["simple_time_bin"] = (simple_df["min_since_midnight"] // 5).astype("int16")

# 1 hour bins: [0..23]
simple_df["one_time_bin"] = (simple_df["min_since_midnight"] // 60).astype("int8")

# This maps 00:00–00:04 → bin 0, 00:05–00:09 → bin 1, …, 23:55–23:59 → bin 287, across all days.
# Grouping by such bins is a standard way to compare flows at the same time‑of‑day across dates.

# Define arm identifiers (5-min bin, sensor, direction) as the arm key
simple_df["simple_arm"] = list(
    zip(
        simple_df["simple_time_bin"],
        simple_df["EQUIPMENTID"],
        simple_df["LANE_BUNDLE_DIRECTION"]
    )
)

# Define arm identifiers: (30‑min bin, sensor, direction)
simple_df["half_arm"] = list(
    zip(
        simple_df["half_time_bin"],
        simple_df["EQUIPMENTID"],
        simple_df["LANE_BUNDLE_DIRECTION"]
    )
)

# Define arm identifiers: (1hr bin, sensor, direction)
simple_df["one_arm"] = list(
    zip(
        simple_df["one_time_bin"],
        simple_df["EQUIPMENTID"],
        simple_df["LANE_BUNDLE_DIRECTION"]
    )
)

In [29]:
simple_df["simple_arm"].nunique()

7488

In [28]:
simple_df["half_arm"].nunique()

1248

In [30]:
simple_df["one_arm"].nunique()

624

In [31]:
simple_df

,EQUIPMENTID,AGG_PERIOD_START,LANE_BUNDLE_DIRECTION,TOTAL_VOLUME,min_since_midnight,half_time_bin,simple_time_bin,one_time_bin,simple_arm,half_arm,one_arm
0,121726,2013-04-12 01:05:00,C,43,65,2,13,1,"(13, 121726, C)","(2, 121726, C)","(1, 121726, C)"
1,121726,2013-04-12 01:05:00,D,60,65,2,13,1,"(13, 121726, D)","(2, 121726, D)","(1, 121726, D)"
2,121726,2013-04-12 01:10:00,C,48,70,2,14,1,"(14, 121726, C)","(2, 121726, C)","(1, 121726, C)"
3,121726,2013-04-12 01:10:00,D,64,70,2,14,1,"(14, 121726, D)","(2, 121726, D)","(1, 121726, D)"
4,121726,2013-04-12 01:15:00,C,37,75,2,15,1,"(15, 121726, C)","(2, 121726, C)","(1, 121726, C)"
...,...,...,...,...,...,...,...,...,...,...,...
7094430,121733,2015-12-31 15:55:00,D,365,955,31,191,15,"(191, 121733, D)","(31, 121733, D)","(15, 121733, D)"
7094431,121741,2015-12-31 15:55:00,C,297,955,31,191,15,"(191, 121741, C)","(31, 121741, C)","(15, 121741, C)"
7094432,121741,2015-12-31 15:55:00,D,359,955,31,191,15,"(191, 121741, D)","(31, 121741, D)","(15, 121741, D)"
7094433,121741,2015-12-31 16:00:00,C,266,960,32,192,16,"(192, 121741, C)","(32, 121741, C)","(16, 121741, C)"


In [32]:
simple_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094435 entries, 0 to 7094434
Data columns (total 11 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   EQUIPMENTID            int64         
 1   AGG_PERIOD_START       datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  object        
 3   TOTAL_VOLUME           int64         
 4   min_since_midnight     int32         
 5   half_time_bin          int8          
 6   simple_time_bin        int16         
 7   one_time_bin           int8          
 8   simple_arm             object        
 9   half_arm               object        
 10  one_arm                object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(2), object(4)
memory usage: 433.0+ MB


In [8]:
simple_df.describe()

,EQUIPMENTID,AGG_PERIOD_START,TOTAL_VOLUME,min_since_midnight,half_time_bin,simple_time_bin,one_time_bin
count,7.094435e+06,7094435,7.094435e+06,7.094435e+06,7.094435e+06,7.094435e+06,7.094435e+06
mean,1.217385e+05,2014-08-08 19:08:26.350710016,1.807496e+02,7.172668e+02,2.349221e+01,1.434534e+02,1.149610e+01
min,1.217260e+05,2013-03-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.217320e+05,2013-11-20 00:00:00,5.800000e+01,3.550000e+02,1.100000e+01,7.100000e+01,5.000000e+00
50%,1.217350e+05,2014-08-18 02:20:00,1.610000e+02,7.150000e+02,2.300000e+01,1.430000e+02,1.100000e+01
75%,1.217420e+05,2015-04-21 20:55:00,2.840000e+02,1.080000e+03,3.600000e+01,2.160000e+02,1.800000e+01
max,1.217560e+05,2015-12-31 23:55:00,8.750000e+02,1.435000e+03,4.700000e+01,2.870000e+02,2.300000e+01
std,9.933554e+00,NaN,1.325153e+02,4.158539e+02,1.385887e+01,8.317078e+01,6.924901e+00


In [10]:
simple_df["TOTAL_VOLUME"].max()

np.int64(875)

In [19]:
half_hour_volume = (
    simple_df.groupby(
        [simple_df["AGG_PERIOD_START"].dt.date,  # optional, keeps days separate
         "half_time_bin",
         "EQUIPMENTID",
         "LANE_BUNDLE_DIRECTION"]
    )["TOTAL_VOLUME"]
    .sum()
)

max_30min_volume = half_hour_volume.max()
print(max_30min_volume)

6240


# Train/Test Split

In [33]:
# Separate the train and test data
cutoff_date = pd.Timestamp("2015-01-01")

# Train data: 2013-2014
simple_train_df = simple_df[simple_df["AGG_PERIOD_START"] < cutoff_date]
# Test data: 2015
simple_test_df  = simple_df[simple_df["AGG_PERIOD_START"] >= cutoff_date]

# Save the data
simple_train_df.to_csv("./Dataset/simple_train.csv", index=False)
simple_test_df.to_csv("./Dataset/simple_test.csv", index=False)
# simple_train_df.to_csv("../Dados/simple_train.csv", index=False)
# simple_test_df.to_csv("../Dados/simple_test.csv", index=False)

In [34]:
simple_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4528737 entries, 0 to 4528736
Data columns (total 11 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   EQUIPMENTID            int64         
 1   AGG_PERIOD_START       datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  object        
 3   TOTAL_VOLUME           int64         
 4   min_since_midnight     int32         
 5   half_time_bin          int8          
 6   simple_time_bin        int16         
 7   one_time_bin           int8          
 8   simple_arm             object        
 9   half_arm               object        
 10  one_arm                object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(2), object(4)
memory usage: 311.0+ MB


In [35]:
simple_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2565698 entries, 4528737 to 7094434
Data columns (total 11 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   EQUIPMENTID            int64         
 1   AGG_PERIOD_START       datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  object        
 3   TOTAL_VOLUME           int64         
 4   min_since_midnight     int32         
 5   half_time_bin          int8          
 6   simple_time_bin        int16         
 7   one_time_bin           int8          
 8   simple_arm             object        
 9   half_arm               object        
 10  one_arm                object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(2), object(4)
memory usage: 176.2+ MB
